<a href="https://colab.research.google.com/github/anitarestrepo16/machine-learning-class-2023/blob/main/MP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.pipeline import make_pipeline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf

In [2]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [80]:
vote_df = pd.read_csv('/drive/My Drive/Colab Notebooks/vote.csv')
vote_df.head()

,prtage,pesex,ptdtrace,pehspnon,prcitshp,peeduca,vote
0,19,FEMALE,White Only,NON-HISPANIC,"NATIVE, BORN IN THE UNITED",SOME COLLEGE BUT NO DEGREE,vote
1,35,MALE,White Only,NON-HISPANIC,"NATIVE, BORN IN THE UNITED","MASTER'S DEGREE (EX: MA, MS,",vote
2,48,MALE,White Only,HISPANIC,"FOREIGN BORN, U.S. CITIZEN BY",5TH OR 6TH GRADE,vote
3,55,MALE,White Only,NON-HISPANIC,"NATIVE, BORN IN THE UNITED",BACHELOR'S DEGREE,did not vote
4,25,FEMALE,White Only,NON-HISPANIC,"NATIVE, BORN IN THE UNITED",SOME COLLEGE BUT NO DEGREE,vote


In [81]:
work_df = pd.read_csv('/drive/My Drive/Colab Notebooks/work.csv')
work_df.head()

,prtage,pesex,ptdtrace,pehspnon,prcitshp,peeduca,work
0,35,MALE,White Only,NON-HISPANIC,"NATIVE, BORN IN THE UNITED",HIGH SCHOOL GRAD-DIPLOMA OR,flexible
1,41,MALE,White Only,NON-HISPANIC,"NATIVE, BORN IN THE UNITED",HIGH SCHOOL GRAD-DIPLOMA OR,flexible
2,53,MALE,White Only,NON-HISPANIC,"NATIVE, BORN IN THE UNITED",BACHELOR'S DEGREE,not flexible
3,21,FEMALE,White Only,NON-HISPANIC,"NATIVE, BORN IN THE UNITED",SOME COLLEGE BUT NO DEGREE,flexible
4,45,FEMALE,Black Only,NON-HISPANIC,"NATIVE, BORN IN THE UNITED",SOME COLLEGE BUT NO DEGREE,flexible


#2a

In [62]:
vote_df["vote"].unique()

array(['vote', 'did not vote'], dtype=object)

In [6]:
work_df["work"].unique()

array(['flexible', 'not flexible'], dtype=object)

In [82]:
# Map labels to 0 and 1
work_mapper = {'flexible': 1, "not flexible": 0}
vote_mapper = {'vote': 1, "did not vote": 0}
work_df['work'] = work_df['work'].replace(work_mapper)
vote_df['vote'] = vote_df['vote'].replace(vote_mapper)

In [64]:
cat_vars = vote_df.columns[1:6].values.tolist()
cat_vars

['pesex', 'ptdtrace', 'pehspnon', 'prcitshp', 'peeduca']

#2b

In [65]:
vote_vals = {}
work_vals = {}
for col in cat_vars:
    vote_vals[col] = vote_df[col].unique()
    work_vals[col] = work_df[col].unique()

print(vote_vals)
print(work_vals)

{'pesex': array(['FEMALE', 'MALE'], dtype=object), 'ptdtrace': array(['White Only', 'Black Only', 'Hawaiian/Pacific Islander Only',
       'Asian Only', 'American Indian, Alaskan', 'White-AI', 'Black-AI',
       'W-B-AI', '2 or 3 Races', 'White-Asian', 'Asian-HP', 'White-Black',
       'White-Hawaiian', 'W-A-HP', 'Black-Asian'], dtype=object), 'pehspnon': array(['NON-HISPANIC', 'HISPANIC'], dtype=object), 'prcitshp': array(['NATIVE, BORN IN THE UNITED', 'FOREIGN BORN, U.S. CITIZEN BY',
       'NATIVE, BORN IN PUERTO RICO OR', 'NATIVE, BORN ABROAD OF'],
      dtype=object), 'peeduca': array(['SOME COLLEGE BUT NO DEGREE', "MASTER'S DEGREE (EX: MA, MS,",
       '5TH OR 6TH GRADE', "BACHELOR'S DEGREE",
       'HIGH SCHOOL GRAD-DIPLOMA OR', 'DOCTORATE DEGREE',
       'ASSOCIATE DEGREE-OCCUPATIONAL/', 'PROFESSIONAL SCHOOL DEG',
       '11TH GRADE', '7TH OR 8TH GRADE', 'ASSOCIATE DEGREE-ACADEMIC',
       '9TH GRADE', '12TH GRADE NO DIPLOMA', '10TH GRADE',
       '1ST, 2ND, 3RD OR 4TH GRADE', 

## Discrepancies:
ptdtrace

In [66]:
missing = []
for i in vote_vals['ptdtrace']:
    if i not in work_vals['ptdtrace']:
        missing.append(i)
missing

['W-B-AI', 'W-A-HP', 'Black-Asian']

In [16]:
missing = []
for i in work_vals['ptdtrace']:
    if i not in vote_vals['ptdtrace']:
        missing.append(i)
missing

['4 or 5 Races']

pehspnon

In [17]:
missing = []
for i in vote_vals['pehspnon']:
    if i not in work_vals['pehspnon']:
        missing.append(i)
missing

[]

In [19]:
missing = []
for i in work_vals['pehspnon']:
    if i not in vote_vals['pehspnon']:
        missing.append(i)
missing

[]

prcitshp

In [20]:
missing = []
for i in vote_vals['prcitshp']:
    if i not in work_vals['prcitshp']:
        missing.append(i)
missing

[]

In [21]:
missing = []
for i in work_vals['prcitshp']:
    if i not in vote_vals['prcitshp']:
        missing.append(i)
missing

['FOREIGN BORN, NOT A CITIZEN OF']

peeduca

In [22]:
missing = []
for i in vote_vals['peeduca']:
    if i not in work_vals['peeduca']:
        missing.append(i)
missing

[]

In [23]:
missing = []
for i in work_vals['peeduca']:
    if i not in vote_vals['peeduca']:
        missing.append(i)
missing

[]

#2c

In [83]:
vote_df = pd.get_dummies(vote_df, columns = cat_vars)
work_df = pd.get_dummies(work_df, columns = cat_vars)

In [68]:
vote_df.columns

Index(['prtage', 'vote', 'pesex_FEMALE', 'pesex_MALE', 'ptdtrace_2 or 3 Races',
       'ptdtrace_American Indian, Alaskan', 'ptdtrace_Asian Only',
       'ptdtrace_Asian-HP', 'ptdtrace_Black Only', 'ptdtrace_Black-AI',
       'ptdtrace_Black-Asian', 'ptdtrace_Hawaiian/Pacific Islander Only',
       'ptdtrace_W-A-HP', 'ptdtrace_W-B-AI', 'ptdtrace_White Only',
       'ptdtrace_White-AI', 'ptdtrace_White-Asian', 'ptdtrace_White-Black',
       'ptdtrace_White-Hawaiian', 'pehspnon_HISPANIC', 'pehspnon_NON-HISPANIC',
       'prcitshp_FOREIGN BORN, U.S. CITIZEN BY',
       'prcitshp_NATIVE, BORN ABROAD OF',
       'prcitshp_NATIVE, BORN IN PUERTO RICO OR',
       'prcitshp_NATIVE, BORN IN THE UNITED', 'peeduca_10TH GRADE',
       'peeduca_11TH GRADE', 'peeduca_12TH GRADE NO DIPLOMA',
       'peeduca_1ST, 2ND, 3RD OR 4TH GRADE', 'peeduca_5TH OR 6TH GRADE',
       'peeduca_7TH OR 8TH GRADE', 'peeduca_9TH GRADE',
       'peeduca_ASSOCIATE DEGREE-ACADEMIC',
       'peeduca_ASSOCIATE DEGREE-OCCUPA

In [69]:
work_df.columns

Index(['prtage', 'work', 'pesex_FEMALE', 'pesex_MALE', 'ptdtrace_2 or 3 Races',
       'ptdtrace_4 or 5 Races', 'ptdtrace_American Indian, Alaskan',
       'ptdtrace_Asian Only', 'ptdtrace_Asian-HP', 'ptdtrace_Black Only',
       'ptdtrace_Black-AI', 'ptdtrace_Hawaiian/Pacific Islander Only',
       'ptdtrace_White Only', 'ptdtrace_White-AI', 'ptdtrace_White-Asian',
       'ptdtrace_White-Black', 'ptdtrace_White-Hawaiian', 'pehspnon_HISPANIC',
       'pehspnon_NON-HISPANIC', 'prcitshp_FOREIGN BORN, NOT A CITIZEN OF',
       'prcitshp_FOREIGN BORN, U.S. CITIZEN BY',
       'prcitshp_NATIVE, BORN ABROAD OF',
       'prcitshp_NATIVE, BORN IN PUERTO RICO OR',
       'prcitshp_NATIVE, BORN IN THE UNITED', 'peeduca_10TH GRADE',
       'peeduca_11TH GRADE', 'peeduca_12TH GRADE NO DIPLOMA',
       'peeduca_1ST, 2ND, 3RD OR 4TH GRADE', 'peeduca_5TH OR 6TH GRADE',
       'peeduca_7TH OR 8TH GRADE', 'peeduca_9TH GRADE',
       'peeduca_ASSOCIATE DEGREE-ACADEMIC',
       'peeduca_ASSOCIATE DEGREE-

#2d

In [84]:
vote_df["prcitshp_FOREIGN BORN, NOT A CITIZEN OF"] = 0
work_df['ptdtrace_W-B-AI'] = 0
work_df['ptdtrace_W-A-HP'] = 0
work_df['ptdtrace_Black-Asian'] = 0
vote_df['ptdtrace_4 or 5 Races'] = 0

#3

In [85]:
c_vals = [0.1, 1, 5, 10]
kernels = ['linear', 'poly', 'sigmoid']
rand_state = 26
cv_folds = 5

In [86]:
X = work_df.drop('work', axis = 1)
y = work_df['work']

In [87]:
pipe = make_pipeline(StandardScaler(), GridSearchCV(estimator = SVC(), param_grid = {"kernel": kernels, "C": c_vals}, cv = KFold(n_splits = cv_folds, shuffle = True, random_state = rand_state)))
pipe.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gridsearchcv',
                 GridSearchCV(cv=KFold(n_splits=5, random_state=26, shuffle=True),
                              estimator=SVC(),
                              param_grid={'C': [0.1, 1, 5, 10],
                                          'kernel': ['linear', 'poly',
                                                     'sigmoid']}))])

In [88]:
all_mods = pd.DataFrame(pipe[1].cv_results_)
all_mods

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.384366,0.018819,0.054141,0.002463,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",0.862,0.864,0.870,0.861,0.850,0.8614,0.006499,3
1,0.972195,0.212375,0.167110,0.044040,0.1,poly,"{'C': 0.1, 'kernel': 'poly'}",0.648,0.632,0.631,0.641,0.605,0.6314,0.014596,12
2,1.005041,0.224252,0.176914,0.037518,0.1,sigmoid,"{'C': 0.1, 'kernel': 'sigmoid'}",0.858,0.862,0.869,0.862,0.850,0.8602,0.006210,5
3,0.480754,0.012662,0.051945,0.000742,1,linear,"{'C': 1, 'kernel': 'linear'}",0.863,0.866,0.869,0.861,0.852,0.8622,0.005776,1
4,0.707239,0.007091,0.115695,0.003683,1,poly,"{'C': 1, 'kernel': 'poly'}",0.843,0.835,0.839,0.835,0.824,0.8352,0.006337,8
5,0.893684,0.104162,0.133112,0.019981,1,sigmoid,"{'C': 1, 'kernel': 'sigmoid'}",0.822,0.827,0.832,0.826,0.827,0.8268,0.003187,9
6,0.792058,0.024294,0.050341,0.000424,5,linear,"{'C': 5, 'kernel': 'linear'}",0.863,0.863,0.869,0.861,0.850,0.8612,0.006210,4
7,0.546280,0.007251,0.086507,0.005447,5,poly,"{'C': 5, 'kernel': 'poly'}",0.857,0.849,0.855,0.852,0.845,0.8516,0.004271,7
8,0.496431,0.086506,0.076830,0.012943,5,sigmoid,"{'C': 5, 'kernel': 'sigmoid'}",0.819,0.816,0.810,0.817,0.820,0.8164,0.003499,10
9,1.292262,0.306557,0.070790,0.022954,10,linear,"{'C': 10, 'kernel': 'linear'}",0.863,0.866,0.869,0.861,0.851,0.8620,0.006132,2


#4

In [89]:
pipe[1].best_params_

{'C': 1, 'kernel': 'linear'}

#5

In [131]:
pipe[1].best_score_

0.8622

In [77]:
pipe[1].scorer_

<function sklearn.metrics._scorer._passthrough_scorer(estimator, *args, **kwargs)>

#6

In [91]:
svm = SVC(C = 1, kernel = 'linear')
pipe2 = make_pipeline(StandardScaler(), svm)
pipe2.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=1, kernel='linear'))])

In [113]:
vote_df.columns

Index(['prtage', 'vote', 'pesex_FEMALE', 'pesex_MALE', 'ptdtrace_2 or 3 Races',
       'ptdtrace_American Indian, Alaskan', 'ptdtrace_Asian Only',
       'ptdtrace_Asian-HP', 'ptdtrace_Black Only', 'ptdtrace_Black-AI',
       'ptdtrace_Black-Asian', 'ptdtrace_Hawaiian/Pacific Islander Only',
       'ptdtrace_W-A-HP', 'ptdtrace_W-B-AI', 'ptdtrace_White Only',
       'ptdtrace_White-AI', 'ptdtrace_White-Asian', 'ptdtrace_White-Black',
       'ptdtrace_White-Hawaiian', 'pehspnon_HISPANIC', 'pehspnon_NON-HISPANIC',
       'prcitshp_FOREIGN BORN, U.S. CITIZEN BY',
       'prcitshp_NATIVE, BORN ABROAD OF',
       'prcitshp_NATIVE, BORN IN PUERTO RICO OR',
       'prcitshp_NATIVE, BORN IN THE UNITED', 'peeduca_10TH GRADE',
       'peeduca_11TH GRADE', 'peeduca_12TH GRADE NO DIPLOMA',
       'peeduca_1ST, 2ND, 3RD OR 4TH GRADE', 'peeduca_5TH OR 6TH GRADE',
       'peeduca_7TH OR 8TH GRADE', 'peeduca_9TH GRADE',
       'peeduca_ASSOCIATE DEGREE-ACADEMIC',
       'peeduca_ASSOCIATE DEGREE-OCCUPA

In [101]:
X.columns

Index(['prtage', 'pesex_FEMALE', 'pesex_MALE', 'ptdtrace_2 or 3 Races',
       'ptdtrace_4 or 5 Races', 'ptdtrace_American Indian, Alaskan',
       'ptdtrace_Asian Only', 'ptdtrace_Asian-HP', 'ptdtrace_Black Only',
       'ptdtrace_Black-AI', 'ptdtrace_Hawaiian/Pacific Islander Only',
       'ptdtrace_White Only', 'ptdtrace_White-AI', 'ptdtrace_White-Asian',
       'ptdtrace_White-Black', 'ptdtrace_White-Hawaiian', 'pehspnon_HISPANIC',
       'pehspnon_NON-HISPANIC', 'prcitshp_FOREIGN BORN, NOT A CITIZEN OF',
       'prcitshp_FOREIGN BORN, U.S. CITIZEN BY',
       'prcitshp_NATIVE, BORN ABROAD OF',
       'prcitshp_NATIVE, BORN IN PUERTO RICO OR',
       'prcitshp_NATIVE, BORN IN THE UNITED', 'peeduca_10TH GRADE',
       'peeduca_11TH GRADE', 'peeduca_12TH GRADE NO DIPLOMA',
       'peeduca_1ST, 2ND, 3RD OR 4TH GRADE', 'peeduca_5TH OR 6TH GRADE',
       'peeduca_7TH OR 8TH GRADE', 'peeduca_9TH GRADE',
       'peeduca_ASSOCIATE DEGREE-ACADEMIC',
       'peeduca_ASSOCIATE DEGREE-OCCUPATI

In [118]:
X_vote = vote_df.drop(['vote'], axis = 1)

In [121]:
X_vote = X_vote[X.columns]
X_vote.columns

Index(['prtage', 'pesex_FEMALE', 'pesex_MALE', 'ptdtrace_2 or 3 Races',
       'ptdtrace_4 or 5 Races', 'ptdtrace_American Indian, Alaskan',
       'ptdtrace_Asian Only', 'ptdtrace_Asian-HP', 'ptdtrace_Black Only',
       'ptdtrace_Black-AI', 'ptdtrace_Hawaiian/Pacific Islander Only',
       'ptdtrace_White Only', 'ptdtrace_White-AI', 'ptdtrace_White-Asian',
       'ptdtrace_White-Black', 'ptdtrace_White-Hawaiian', 'pehspnon_HISPANIC',
       'pehspnon_NON-HISPANIC', 'prcitshp_FOREIGN BORN, NOT A CITIZEN OF',
       'prcitshp_FOREIGN BORN, U.S. CITIZEN BY',
       'prcitshp_NATIVE, BORN ABROAD OF',
       'prcitshp_NATIVE, BORN IN PUERTO RICO OR',
       'prcitshp_NATIVE, BORN IN THE UNITED', 'peeduca_10TH GRADE',
       'peeduca_11TH GRADE', 'peeduca_12TH GRADE NO DIPLOMA',
       'peeduca_1ST, 2ND, 3RD OR 4TH GRADE', 'peeduca_5TH OR 6TH GRADE',
       'peeduca_7TH OR 8TH GRADE', 'peeduca_9TH GRADE',
       'peeduca_ASSOCIATE DEGREE-ACADEMIC',
       'peeduca_ASSOCIATE DEGREE-OCCUPATI

In [122]:
vote_df['imputed_work'] = pipe2.predict(X_vote)

In [123]:
vote_df['imputed_work'].describe()

count    5000.000000
mean        0.549400
std         0.497603
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: imputed_work, dtype: float64

#7

In [124]:
result = smf.ols('vote ~  imputed_work + prtage + np.power(prtage, 2) + pesex_FEMALE', data = vote_df).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   vote   R-squared:                       0.567
Model:                            OLS   Adj. R-squared:                  0.567
Method:                 Least Squares   F-statistic:                     1637.
Date:                Tue, 07 Mar 2023   Prob (F-statistic):               0.00
Time:                        21:52:46   Log-Likelihood:                -1528.5
No. Observations:                5000   AIC:                             3067.
Df Residuals:                    4995   BIC:                             3100.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               1.1090    

#8

In [109]:
def M(a, b):
    return 1/(1-2*b)*(1-((1-b)*b/a)-((1-b)*b/(1-a)))

In [130]:
prop_flexible = sum(vote_df["imputed_work"] == 1)/len(vote_df['imputed_work'])
prop_flexible

0.5494

In [138]:
cv_error = 1 - pipe[1].best_score_
cv_error

0.13780000000000003

In [139]:
scale_bias = M(prop_flexible, cv_error)
scale_bias

0.7179328513849043

#8

In [140]:
estimate = result.params[1]
estimate

0.32127927000276363

In [141]:
estimate/scale_bias

0.4475060158941197

Bias correction makes the estimated beta larger!